# BharatViz API - Jupyter Notebook Example

This notebook demonstrates how to use the BharatViz API to generate India state-level choropleth maps directly from Python.

## Prerequisites

```bash
pip install requests pandas matplotlib pillow
```

## API Endpoint

- **Local Development**: http://localhost:3001/api/v1/states/map
- **Production**: http://bharatviz.saketlab.in/api/v1/states/map

In [ ]:
import requests
import pandas as pd
import base64
from io import BytesIO
from PIL import Image
import matplotlib.pyplot as plt

# API Configuration
API_URL = "http://localhost:3001/api/v1/states/map"

## Example 1: Basic Usage with Sample Data

In [ ]:
# Sample data: State-wise literacy rates
data = [
    {"state": "Kerala", "value": 96.2},
    {"state": "Lakshadweep", "value": 91.8},
    {"state": "Mizoram", "value": 91.3},
    {"state": "Goa", "value": 88.7},
    {"state": "Tripura", "value": 87.2},
    {"state": "Himachal Pradesh", "value": 82.8},
    {"state": "Maharashtra", "value": 82.3},
    {"state": "Sikkim", "value": 81.4},
    {"state": "Tamil Nadu", "value": 80.1},
    {"state": "Nagaland", "value": 79.6},
    {"state": "Gujarat", "value": 78.0},
    {"state": "Manipur", "value": 76.9},
    {"state": "Punjab", "value": 75.8},
    {"state": "Haryana", "value": 75.6},
    {"state": "Karnataka", "value": 75.4},
    {"state": "Uttarakhand", "value": 78.8},
    {"state": "West Bengal", "value": 76.3},
    {"state": "Odisha", "value": 72.9},
    {"state": "Andhra Pradesh", "value": 67.4},
    {"state": "Assam", "value": 72.2},
    {"state": "Meghalaya", "value": 74.4},
    {"state": "Madhya Pradesh", "value": 69.3},
    {"state": "Chhattisgarh", "value": 70.3},
    {"state": "Uttar Pradesh", "value": 67.7},
    {"state": "Jharkhand", "value": 66.4},
    {"state": "Bihar", "value": 61.8},
    {"state": "Rajasthan", "value": 66.1},
    {"state": "Arunachal Pradesh", "value": 65.4},
]

# API Request
request_body = {
    "data": data,
    "colorScale": "viridis",
    "mainTitle": "Literacy Rate by State",
    "legendTitle": "Literacy Rate (%)",
    "formats": ["png"],  # Request PNG format
}

response = requests.post(API_URL, json=request_body)
result = response.json()

print(f"API Response: {result['success']}")
print(f"Data points: {result['metadata']['dataPoints']}")
print(
    f"Min: {result['metadata']['minValue']:.2f}, Max: {result['metadata']['maxValue']:.2f}, Mean: {result['metadata']['meanValue']:.2f}"
)

In [ ]:
# Display the generated map
png_data = result["exports"][0]["data"]
image_bytes = base64.b64decode(png_data)
image = Image.open(BytesIO(image_bytes))

plt.figure(figsize=(12, 12))
plt.imshow(image)
plt.axis("off")
plt.title("Generated Map", fontsize=16, pad=20)
plt.tight_layout()
plt.show()

## Example 2: Load Data from Pandas DataFrame

In [ ]:
# Create a DataFrame
df = pd.DataFrame(
    {
        "state_name": [
            "Maharashtra",
            "Karnataka",
            "Tamil Nadu",
            "Gujarat",
            "West Bengal",
            "Uttar Pradesh",
            "Rajasthan",
            "Bihar",
            "Madhya Pradesh",
            "Kerala",
        ],
        "gdp_per_capita": [2.75, 2.45, 2.58, 2.32, 1.54, 0.68, 1.42, 0.48, 1.08, 2.12],
    }
)

# Convert to API format
data = df.rename(columns={"state_name": "state", "gdp_per_capita": "value"}).to_dict(
    "records"
)

# Generate map
request_body = {
    "data": data,
    "colorScale": "plasma",
    "mainTitle": "GDP Per Capita by State",
    "legendTitle": "GDP Per Capita (Lakh ₹)",
    "formats": ["png", "svg", "pdf"],  # Get all formats
}

response = requests.post(API_URL, json=request_body)
result = response.json()

# Display PNG
png_export = next(e for e in result["exports"] if e["format"] == "png")
image = Image.open(BytesIO(base64.b64decode(png_export["data"])))

plt.figure(figsize=(12, 12))
plt.imshow(image)
plt.axis("off")
plt.show()

## Example 3: Save Maps to Files

In [ ]:
# Save all formats to files
for export in result["exports"]:
    filename = f"india_map.{export['format']}"

    # Decode base64 and save
    file_data = base64.b64decode(export["data"])

    with open(filename, "wb") as f:
        f.write(file_data)

    print(f"Saved: {filename} ({len(file_data) / 1024:.2f} KB)")

## Example 4: Different Color Scales

In [ ]:
# Available color scales
color_scales = ["spectral", "viridis", "plasma", "inferno", "blues", "reds", "greens"]

# Sample data
sample_data = [
    {"state": "Maharashtra", "value": 75},
    {"state": "Karnataka", "value": 85},
    {"state": "Tamil Nadu", "value": 80},
    {"state": "Gujarat", "value": 65},
    {"state": "Kerala", "value": 95},
]

# Generate maps with different color scales
fig, axes = plt.subplots(2, 4, figsize=(20, 10))
axes = axes.flatten()

for idx, color_scale in enumerate(color_scales):
    request_body = {
        "data": sample_data,
        "colorScale": color_scale,
        "mainTitle": f"{color_scale.title()} Scale",
        "legendTitle": "Value",
        "formats": ["png"],
    }

    response = requests.post(API_URL, json=request_body)
    result = response.json()

    png_data = result["exports"][0]["data"]
    image = Image.open(BytesIO(base64.b64decode(png_data)))

    axes[idx].imshow(image)
    axes[idx].axis("off")
    axes[idx].set_title(color_scale.title(), fontsize=14)

# Hide the last unused subplot
axes[-1].axis("off")

plt.tight_layout()
plt.show()

## Example 5: Minimal Map (No Labels)

In [ ]:
# Generate a minimal map without state names and values
request_body = {
    "data": sample_data,
    "colorScale": "spectral",
    "hideStateNames": True,
    "hideValues": True,
    "mainTitle": "Clean Map",
    "legendTitle": "Value",
    "formats": ["png"],
}

response = requests.post(API_URL, json=request_body)
result = response.json()

png_data = result["exports"][0]["data"]
image = Image.open(BytesIO(base64.b64decode(png_data)))

plt.figure(figsize=(12, 12))
plt.imshow(image)
plt.axis("off")
plt.show()

## Example 6: Load from CSV File

## Helper Function: Easy Map Generation

In [ ]:
def generate_india_map(
    data,
    title="BharatViz",
    legend_title="Values",
    color_scale="spectral",
    show=True,
    save_path=None,
):
    """
    Generate India state map using BharatViz API

    Parameters:
    - data: list of dict with 'state' and 'value' keys, or pandas DataFrame
    - title: Main title for the map
    - legend_title: Title for the color legend
    - color_scale: Color scale to use
    - show: Whether to display the map
    - save_path: Path to save the PNG file (optional)

    Returns:
    - Image object
    """
    # Convert DataFrame to list of dicts if needed
    if isinstance(data, pd.DataFrame):
        data = data.to_dict("records")

    # API request
    request_body = {
        "data": data,
        "colorScale": color_scale,
        "mainTitle": title,
        "legendTitle": legend_title,
        "formats": ["png"],
    }

    response = requests.post(API_URL, json=request_body)
    result = response.json()

    if not result["success"]:
        raise Exception(f"API error: {result.get('error', 'Unknown error')}")

    # Get image
    png_data = result["exports"][0]["data"]
    image = Image.open(BytesIO(base64.b64decode(png_data)))

    # Save if requested
    if save_path:
        image.save(save_path)
        print(f"Map saved to: {save_path}")

    # Display if requested
    if show:
        plt.figure(figsize=(12, 12))
        plt.imshow(image)
        plt.axis("off")
        plt.show()

    return image


# Example usage
my_data = [
    {"state": "Maharashtra", "value": 112.3},
    {"state": "Gujarat", "value": 98.5},
    {"state": "Karnataka", "value": 85.7},
]

img = generate_india_map(
    data=my_data,
    title="My Custom Map",
    legend_title="My Metric",
    color_scale="viridis",
    save_path="my_map.png",
)